# 프롬프트 템플릿을 사용하여 베드락 모델 호출하기

> *이 노트북은 SageMaker Studio*의 `Data Science 3.0` 커널과 잘 작동해야 합니다.

## 소개

이 노트북에서는 리테일 고객이 홈페이지에 제품에 대한 설명을 생상하는 방법을 보여드립니다. 다른 제품에 대한 설명을 예시로 포함하여 모델에 추가 컨텍스트를 제공합니다. 이로 인해 제로 샷 프롬프트를 통해 생성된 컨텐츠보다 품질과 관련성이 훨씬 더 우수합니다. 

[LangChain](https://python.langchain.com/docs/get_started/introduction.html)은 언어 모델로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. 이 프레임워크의 핵심 측면을 통해 다양한 구성 요소를 연결하여 고급 사용 사례를 만들어 대규모 언어 모델을 보강할 수 있습니다.

이 노트북에서는 LangChain에서 제공하는 Bedrock API를 사용하겠습니다. 이 예제에서 사용된 프롬프트는 텍스트 생성 요청에 컨텍스트를 추가하기 위한 사용자 정의 LangChain 프롬프트 템플릿을 생성합니다.

**참고:** *이 노트북은 AWS 환경 내부 또는 외부에서 실행할 수 있습니다*.

#### 컨텍스트

이 노트북에서는 LangChain 프레임워크 내에서 Amazon Bedrock과 통합하여 사용는 방법과 PromptTemplate의 도움으로 텍스트를 생성하는 데 어떻게 사용될 수 있는지 살펴보겠습니다.

#### 패턴

작업, 명령어로 구성된 입력과 모델 내부에서 출력을 생성하기 위한 입력, 두가지로 구성된 Amazon Bedrock API의 LangChain 구현을 간단히 제공하겠습니다. 이 패턴의 목적은 강력한 LLM이 어떻게 당면한 작업을 쉽게 이해하고 매력적인 출력을 생성하는지 보여주기 위함입니다.

![bedrock_langchian_image](./image/bedrock_langchain.jpg)

#### 사용사례
Amazon Bedrock에서 모델의 텍스트 생성 기능을 설명하기 위해 리테일 제품의 설명 생성의 사용 사례를 살펴보겠습니다.

#### 페르소나

리테일 회사인 애니컴퍼니의 제품 담당 매니저인 "김소매"는 하루에도 수십개 이상 신규로 등록되는 제품을 홈페이지에 등록해야 합니다. 제품이 어떤 특징을 가지고 있는지는 납품 업체를 통해 전달받았지만, 이를 고객이 이해하기 쉽게 문장으로 작성하는데는 많은 시간이 소요되었습니다. 이에, 제품의 특징은 다 포함하면서 고객들이 이해하기 쉬운 대량의 제품 설명을 생성하려면 LLM의 도움이 필요합니다.

#### 구현 방법

이 사용 사례를 보여주기 위해 이 노트북에서는 고객의 이전 제품 설명을 기반으로 신규 제품 설명을 생성하는 방법을 보여드리며, Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Anthropic Claude 모델을 사용하겠습니다.


## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) 베드락에 연결해야 합니다.

우선 사전에 설치가 필요한 패키지들을 설치하세요. 그 이후에 셋업에 필요한 라이브러리들을 설치합니다. 

In [17]:
%pip install -r ./dependencies/requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.31.70 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
from langchain.llms.bedrock import Bedrock
from langchain.prompts import PromptTemplate

In [26]:
import json
import sys
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## 베드락 LLM 모델 호출하기

LLM에서 Bedrock 클래스의 인스턴스를 생성하는 것으로 시작하겠습니다. 여기에는 Amazon Bedrock에서 사용할 수 있는 모델의 ARN인 model_id가 필요합니다.

선택적으로 이전에 생성한 boto3 클라이언트를 전달할 수 있으며, `temperature`, `topP`, `maxTokenCount` 또는 `stopSequences`와 같은 매개 변수를 보유할 수 있는 일부 `model_kwargs`도 전달할 수 있습니다(매개 변수에 대한 자세한 내용은 Amazon Bedrock 콘솔에서 살펴볼 수 있습니다).

Amazon Bedrock에서 사용 가능한 텍스트 생성 모델 ID에 대한 [설명서](https://docs.aws.amazon.com/ko_kr/bedrock/latest/userguide/model-ids-arns.html)를 확인하세요.

모델마다 지원하는 `model_kwargs`가 다르다는 점에 유의하세요.

In [27]:
from langchain.llms.bedrock import Bedrock

model_kwargs = {'max_tokens_to_sample':1024, 
                "temperature":0,
                "top_k":250,
                "top_p":1,
                "stop_sequences": ["\n\nHuman:"]
                }

llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = model_kwargs 
                    )

## LangChain 사용자 지정 프롬프트 템플릿 생성

프롬프트에 대한 템플릿을 생성하면 실행할 때마다 다른 입력 변수를 전달할 수 있습니다. 이는 데이터베이스에서 가져올 수 있는 다양한 입력 변수를 사용하여 콘텐츠를 생성해야 할 때 유용합니다. 다음 셀에서는 PromptTemplate을 만드는 방법을 살펴봅니다.

In [29]:
from langchain.prompts import PromptTemplate

# Create a prompt template that has multiple input variables
multi_var_prompt = PromptTemplate(
    input_variables = ["keyword", "productType"],
    template="""
    \n\nHuman: 
    <keywords> 뚜껑이 달린 퇴비화 가능한 컵, 아이스 커피 컵, 빨대없는 뚜껑, \
    뚜껑이 달린 생분해성 컵, 생분해 성 컵, 커피, 스무디, 음료, 이동 중, 일회용 컵, 친환경, 식물 기반</keywords>
    <command> 위에 나열된 keywords를 사용하여 일회용 컵에 대한 설명을 작성하세요.</command>
    \n\nAssistant: 편리함과 친환경이 드디어 만났습니다! \
    뚜껑이 있는 일회용 스무디 컵은 100% 식물로 만들어집니다. \
    산업 퇴비 시설로 가져가세요.이 퇴비화 가능한 차가운 컵은 모든 종류의 음료에 적합합니다.\
    재사용 가능한 빨대는 훌륭하지만 휴대하기에는 번거롭습니다. \
    스냅 오픈 뚜껑이 있는 빨대 없는 컵은 적당한 크기의 빨대 구멍이 있습니다. \
    새지 않는 텀블러는 아침에 스무디를 마실 때 사용하면 지구에도 좋을지 모르지만 \
    세척하는 데 시간이 더 걸립니다. 저희도 이해합니다, \
    저희도 여기 지구에 살고 있으니까요.\
    대신에 컴포저블 컵을 사용해보시는 건 어떨까요? \
    회사, 학교, 운전할 때 식물성 컵을 가지고 다니세요.

    \n\nHuman:
    <keywords> ${keyword}</keywords>
    <command> 위에 나열된 ${keyword}를 사용하여 ${productType}에 대한 설명을 작성하세요 $는 빼고 작성하세요.</command>
    \n\nAssistant:
    """
)

## 다시 호출
프롬프트 템플릿을 사용하여 호출하면 선별된 응답을 기대할 수 있습니다.

In [30]:
# Pass in values to the input variables

prompt = multi_var_prompt.format(keyword="편광, 디자이너, 컴포터블, 자외선 차단, 에비에이터", productType="썬글라스")

prompt_outcome = llm.predict(prompt)

print(prompt_outcome)

 편안함과 스타일이 만나다!

이 디자이너 선글라스는 최고의 자외선 차단과 편안함을 제공합니다. 

편광 렌즈는 반사광과 눈부심을 줄여주어 시력을 보호합니다. 

컴퍼터블 프레임은 얼굴에 편안하게 맞습니다. 

에비에이터 스타일 프레임은 클래식하면서도 모던한 느낌을 연출합니다.

이 선글라스는 운전, 야외 활동, 휴가 등 다양한 상황에서 스타일리시하고 기능적인 보호를 제공합니다. 

눈부심 없이 선명한 시야를 원하신다면, 이 디자이너 선글라스가 최고의 선택이 될 것입니다.


## Streamlit 어플리케이션 수행하기
위의 내용을 list_app.py로 작성하여 Streamlit 어플리케이션을 수행해보겠습니다. 
터미널 창을 열고 아래 명령어를 수행합니다.

복사하기 : streamlit run list_app.py --server.port 8501
- ![steramlit_example](image/terminal.png)



#### 서비스 확인 방법

1. 본인이 사용하고 있는 스튜디오 도메인 URL을 카피합니다. 보통은 아래과 같을 것입니다.
*  예시  : https://d-XXXXXXXXX.studio.us-east-1.sagemaker.aws/jupyter/default/lab/

2. 그 이후에 스트림릿 앱을 위한 URL로 수정합니다. 
* 위의 **lab**을 **proxy/8080/webapp**으로 변경합니다. 
* 예시 : https://d-XXXXXXXXX.studio.us-east-1.sagemaker.aws/jupyter/default/lab/ --> https://d-XXXXXXXXX.studio.us-east-1.sagemaker.aws/jupyter/default/proxy/8080/webapp

3. 새로운 창을 열어 **2번**에서 수정한 URL을 붙여넣고 실행시킵니다. 

- ![steramlit_example](image/streamlitoutcome.png)

4. 아래 키워드를 입력하고 수행합니다.
- ![steramlit_example](image/streamlitoutput.png)

